link to HF tutorial:
https://huggingface.co/blog/fine-tune-whisper

In [50]:
from datasets import load_dataset, DatasetDict, Audio

from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

import evaluate
metric = evaluate.load("wer")

In [25]:
dataset = DatasetDict()

dataset['train'] = load_dataset('rodoggx/ATCO2-ASR-1h', split='train')
dataset['test'] = load_dataset('rodoggx/ATCO2-ASR-1h', split='test')

In [37]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="english", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
input_str = dataset['train'][0]["text Str"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print("Input text:", input_str)
print("Labels:", labels)
print("Decoded with special tokens:", decoded_with_special)
print("Decoded without special tokens:", decoded_str)
print('is equal:', input_str == decoded_str)

Input text: Turkish Three Whiskey Quebec taxi via echo and echo seven hold short of runway one six
Labels: [50258, 50259, 50359, 50363, 51, 374, 74, 742, 6244, 41132, 4119, 38903, 18984, 5766, 14300, 293, 14300, 3407, 1797, 2099, 295, 26642, 472, 2309, 50257]
Decoded with special tokens: <|startoftranscript|><|en|><|transcribe|><|notimestamps|>Turkish Three Whiskey Quebec taxi via echo and echo seven hold short of runway one six<|endoftext|>
Decoded without special tokens: Turkish Three Whiskey Quebec taxi via echo and echo seven hold short of runway one six
is equal: True


In [26]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [29]:
def prepare_datset(batch):
    audio = batch["audio"]
    batch["input_values"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch['labels'] = tokenizer(batch['text Str']).input_ids
    return batch

dataset = dataset.map(prepare_datset, remove_columns=dataset.column_names['train'], batch_size=8, num_proc=4)

Map (num_proc=4):   0%|          | 0/448 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/112 [00:00<?, ? examples/s]

In [33]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

model.generation_config.language = "english"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [34]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [40]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [41]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels = pred.label_ids
    labels[labels == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels, skip_special_tokens=True)
    wer = metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

In [51]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-en",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [48]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

max_steps is given, it will override any value given in num_train_epochs


In [52]:
trainer.train()

AttributeError: `AcceleratorState` object has no attribute `distributed_type`. This happens if `AcceleratorState._reset_state()` was called and an `Accelerator` or `PartialState` was not reinitialized.